In [ ]:
import numpy as np
import pandas as pd
import spacy
import string
import gensim
import operator
import re

In [ ]:
csv_file = '/content/drive/MyDrive/songLyrics.csv'

df_lyrics = pd.read_csv(csv_file, engine='python', on_bad_lines='warn')

Skipping line 78462: unexpected end of data


In [ ]:
df_lyrics

,song,year,artist,genre,lyrics
0,ego-remix,2009-01-01,beyonce-knowles,Pop,"Oh baby, how you doing? You know I'm gonna cut..."
1,then-tell-me,2009-01-01,beyonce-knowles,Pop,"playin' everything so easy, it's like you seem..."
2,honesty,2009-01-01,beyonce-knowles,Pop,If you search For tenderness It isn't hard to ...
3,you-are-my-rock,2009-01-01,beyonce-knowles,Pop,"Oh oh oh I, oh oh oh I [Verse 1:] If I wrote a..."
4,black-culture,2009-01-01,beyonce-knowles,Pop,"Party the people, the people the party it's po..."
...,...,...,...,...,...
78455,no-surprise,2011-01-01,chris-daughtry,Rock,"I've practiced this for hours, gone round and ..."
78456,ride-it,2007-01-01,geri-halliwell,Pop,I'll keep riding it I'll keep riding it Want t...
78457,so-i-give-up-on-love,2007-01-01,geri-halliwell,Pop,"Good evening ladies and gentlemen, And boys yo..."
78458,half-of-me,2013-01-01,geri-halliwell,Pop,"Just like an apple, Cut down the middle I only..."


In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

spacy_nlp = spacy.load('en_core_web_sm')

#create list of punctuations and stopwords
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

#function for data cleaning and processing
#This can be further enhanced by adding / removing reg-exps as desired.

def spacy_tokenizer(sentence):

    #remove distracting single quotes
    sentence = re.sub('\'','',sentence)

    #remove digits adnd words containing digits
    sentence = re.sub('\w*\d\w*','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)

    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)

    #remove punctunations
    sentence = re.sub(r'[^\w\s]',' ',sentence)

    #creating token object
    tokens = spacy_nlp(sentence)

    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]

    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]

    #return tokens
    return tokens

In [ ]:
lyrics_plot = df_lyrics['lyrics']
lyrics_plot = lyrics_plot.to_numpy()
lyrics_plot = np.split(lyrics_plot, 5)
lyrics_plot

[array(["Oh baby, how you doing? You know I'm gonna cut right to the chase Some women were made but me, myself I like to think that I was created for a special purpose You know, what's more special than you? You feel me It's on baby, let's get lost You don't need to call into work 'cause you're the boss For real, want you to show me how you feel I consider myself lucky, that's a big deal Why? Well, you got the key to my heart But you ain't gonna need it, I'd rather you open up my body And show me secrets, you didn't know was inside No need for me to lie It's too big, it's too wide It's too strong, it won't fit It's too much, it's too tough He talk like this 'cause he can back it up He got a big ego, such a huge ego I love his big ego, it's too much He walk like this 'cause he can back it up Usually I'm humble, right now I don't choose You can leave with me or you could have the blues Some call it arrogant, I call it confident You decide when you find on what I'm working with Damn I kno

In [ ]:
from gensim import corpora

#creating term dictionary
%time dictionary = corpora.Dictionary(lyrics_plot)

#filter out terms which occurs in less than 4 documents and more than 20% of the documents.
#NOTE: Since we have smaller dataset, we will keep this commented for now.

#dictionary.filter_extremes(no_below=4, no_above=0.2)

#list of few which which can be further removed
stoplist = set('Oh baby')
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)


CPU times: user 459 ms, sys: 8.68 ms, total: 468 ms
Wall time: 627 ms


In [ ]:
#print top 10 items from the dictionary with their unique token-id
dict_tokens = [[[dictionary[key], dictionary.token2id[dictionary[key]]] for key, value in dictionary.items() if key <= 10]]
print (dict_tokens)

[[[' Das letze Lied auf der Party das letzte Lied auf der Welt wir liegen uns in den Armen weil uns sonst nichts mehr hÃ¤lt und Respekt an den DJ drauÃ\x9fen wird es schon hell das letzte Lied auf der Party sensationell Dann fang ich doch einfach mal an alle Mann, ohne SpaÃ\x9f kein Fun die Welt kommt mir vor, wie in Technicolor ich bin ein Superpunk, ich bin Rihanna ich bin Drake, ich bin Sly & The Family Stone ich bin Extrabreit, forever young bin einer von 3 Millionen ich umarme dich wahrscheinlich ein bisschen zu doll denn wir beide sind wahrscheinlich ein bisschen zu voll wir sind beide schwer verliebt in diesen Beat, in die Musik dirty motherfuckers mit Herzen aus Gold meine neuen Turnschuh glÃ¤nzen im Licht bin kurz etwas traurig, doch dann kommt der Hit Das letze Lied auf der Party das letzte Lied auf der Welt wir liegen uns in den Armen weil uns sonst nichts mehr hÃ¤lt und Respekt an den DJ drauÃ\x9fen wird es schon hell das letzte Lied auf der Party sensationell Heute Nacht w

In [ ]:
corpus = [dictionary.doc2bow(desc) for desc in lyrics_plot]

word_frequencies = [[(dictionary[id], frequency) for id, frequency in line] for line in corpus[0:1]]

print(word_frequencies)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
%time lyrics_tfidf_model = gensim.models.TfidfModel(corpus, id2word=dictionary)
%time lyrics_lsi_model = gensim.models.LsiModel(lyrics_tfidf_model[corpus], id2word=dictionary, num_topics=300)

CPU times: user 381 ms, sys: 12.1 ms, total: 393 ms
Wall time: 996 ms
CPU times: user 24.2 s, sys: 3.63 s, total: 27.9 s
Wall time: 32 s


In [ ]:
%time gensim.corpora.MmCorpus.serialize('lyrics_tfidf_model_mm', lyrics_tfidf_model[corpus])
%time gensim.corpora.MmCorpus.serialize('lyrics_lsi_model_mm',lyrics_lsi_model[lyrics_tfidf_model[corpus]])

CPU times: user 569 ms, sys: 98.2 ms, total: 667 ms
Wall time: 575 ms
CPU times: user 386 ms, sys: 5.77 ms, total: 392 ms
Wall time: 392 ms


In [ ]:
#Load the indexed corpus
lyrics_tfidf_corpus = gensim.corpora.MmCorpus('lyrics_tfidf_model_mm')
lyrics_lsi_corpus = gensim.corpora.MmCorpus('lyrics_lsi_model_mm')

print(lyrics_tfidf_corpus)
print(lyrics_lsi_corpus)

MmCorpus(5 documents, 78460 features, 78460 non-zero entries)
MmCorpus(5 documents, 5 features, 25 non-zero entries)


In [ ]:
from gensim.similarities import MatrixSimilarity

%time lyrics_index = MatrixSimilarity(lyrics_lsi_corpus, num_features = lyrics_lsi_corpus.num_terms)

CPU times: user 1.85 ms, sys: 0 ns, total: 1.85 ms
Wall time: 1.71 ms


In [ ]:
from operator import itemgetter

def search_similar_lyrics(search_term):

    query_bow = dictionary.doc2bow(spacy_tokenizer(search_term))
    query_tfidf = lyrics_tfidf_model[query_bow]
    query_lsi = lyrics_lsi_model[query_tfidf]

    lyrics_index.num_best = 5

    lyrics_list = lyrics_index[query_lsi]

    lyrics_list.sort(key=itemgetter(1), reverse=True)
    lyrics_names = []

    for j, lyrics in enumerate(lyrics_list):

        lyrics_names.append (
            {
                'song': round((lyrics[1] * 100),2),
                'artist': df_lyrics['lyrics'][lyrics[0]],
                'genre': df_lyrics['song'][lyrics[0]]
            }

        )
        if j == (lyrics_index.num_best-1):
            break

    return pd.DataFrame(df_lyrics, columns=['song','artist','genre', 'lyrics'])

In [ ]:
# search for song titles that are related to below search parameters
function_applied = search_similar_lyrics('Hear')
query = function_applied[function_applied['lyrics'].str.contains("Hear")]
print(query)

                               song  ...                                             lyrics
137                         freedom  ...  Tryna rain, tryna rain on the thunder Tell the...
267               warm-winter-night  ...  There's noone I'd rather be with on this winte...
268                   here-with-you  ...  To all my friends The night is young The music...
284                       jerusalem  ...  Where the mothers of sonsnever cease their wee...
445                        love-for  ...  We from side to season sun and moon ever full ...
...                             ...  ...                                                ...
78018                   hear-me-out  ...  I was drowning in the rain Getting lost beneat...
78098                    crazy-fool  ...  Hi, how you Bet you didn't even think I knew A...
78120                  love-is-pain  ...  What you are What you see It don't matter to m...
78156  the-last-song-i-ll-ever-sing  ...  I'm inviting you to my vaudeville "Oui